In [1]:
import os
import glob
import pandas as pd

In [2]:
path = "raw_csv_files/"

all_files = glob.glob(os.path.join(path, "*.csv"))
print(len(all_files))

45


### Checking the header information

* I took the earliest date and dropped in the headers in "expected_headers"
* Next, I iterated through each of the csv files and compared the list of headers to that exected headers list
* From the dataset it appears that the columns change starting in Feb-2021

In [3]:
expected_headers = ["tripduration","starttime","stoptime","start station id","start station name","start station latitude","start station longitude","end station id","end station name","end station latitude","end station longitude","bikeid","usertype","birth year","gender"]
headers_df = pd.DataFrame(columns=expected_headers)
print(headers_df)

Empty DataFrame
Columns: [tripduration, starttime, stoptime, start station id, start station name, start station latitude, start station longitude, end station id, end station name, end station latitude, end station longitude, bikeid, usertype, birth year, gender]
Index: []


In [4]:
for file in all_files:
    key = file
    df = pd.read_csv(file)
    
    #Expected values to receive in CSV file
    header_list = expected_headers

    import_headers = df.axes[1] #==> 1 is to identify columns
#     print(import_headers)

    
    a = [i for i in import_headers if i not in header_list]
    if len(a) == 0:
        print("Headers match")
    else:
        print(key)

Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
Headers match
raw_csv_files\JC-202102-citibike-tripdata.csv
raw_csv_files\JC-202103-citibike-tripdata.csv
raw_csv_files\JC-202104-citibike-tripdata.csv
raw_csv_files\JC-202105-citibike-tripdata.csv
raw_csv_files\JC-202106-citibike-tripdata.csv
raw_csv_files\JC-202107-citibike-tripdata.csv
raw_csv_files\JC-202108-citibike-tripdata.csv
raw_csv_files\JC-202109-citibike-tripdata.csv


In [5]:
# Getting the file paths for those newly configured file formats
path = "raw_csv_files/"
new_style_df = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "JC-20210[2-9]*.csv"))))

In [6]:
# Just verifying I actually have data here
print(len(new_style_df))

435888


In [7]:
# checking the headers, I know some will need to be renamed and/or removed to have a clean file
new_style_df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

The data is missing the tripduration, which in the previous file generations was in seconds (according to the citibike website). I am going to make sure the datetime is right and then create the tripduration

In [8]:
new_style_df["started_at"] = pd.to_datetime(new_style_df["started_at"])
new_style_df['ended_at'] = pd.to_datetime(new_style_df['ended_at'])

In [9]:
new_style_df['tripduration'] = (new_style_df['ended_at']-new_style_df["started_at"]).dt.total_seconds()

In [10]:
new_style_df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'tripduration'],
      dtype='object')

In [11]:
# new dataframe with just the data I want
new_style_trim = new_style_df[['tripduration','started_at', 'ended_at','start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]

In [12]:
# cleaning the column names to match the old style (which truthfully the new style is better, but it's less data and idk)
final_cut = new_style_trim.rename(columns = {
    'tripduration':'tripduration',
    'started_at':'starttime',
    'ended_at':'stoptime',
    'start_station_id':'start station id',
    'start_station_name':'start station name',
    'start_lat':'start station latitude',
    'start_lng':'start station longitude',
    'end_station_id':'end station id',
    'end_station_name':'end station name',
    'end_lat':'end station latitude',
    'end_lng':'end station longitude'})


In [13]:
final_cut

,tripduration,starttime,stoptime,start station name,start station id,end station name,end station id,start station latitude,start station longitude,end station latitude,end station longitude
0,420.0,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444
1,1355.0,2021-02-27 16:34:05,2021-02-27 16:56:40,Newport Pkwy,JC008,Marin Light Rail,JC013,40.728744,-74.032108,40.714584,-74.042817
2,381.0,2021-02-26 23:16:04,2021-02-26 23:22:25,Journal Square,JC103,Baldwin at Montgomery,JC020,40.733670,-74.062500,40.723659,-74.064194
3,1459.0,2021-02-24 16:51:50,2021-02-24 17:16:09,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964
4,30.0,2021-02-24 20:44:16,2021-02-24 20:44:46,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964
...,...,...,...,...,...,...,...,...,...,...,...
97285,2142.0,2021-09-23 19:08:20,2021-09-23 19:44:02,Harborside,JC104,Union St,JC051,40.719252,-74.034234,40.718211,-74.083639
97286,2933.0,2021-09-10 15:47:24,2021-09-10 16:36:17,8 St & Washington St,HB603,Union St,JC051,40.745984,-74.028199,40.718211,-74.083639
97287,1540.0,2021-09-30 13:28:29,2021-09-30 13:54:09,Harborside,JC104,Union St,JC051,40.719252,-74.034234,40.718211,-74.083639
97288,2833.0,2021-09-14 03:31:22,2021-09-14 04:18:35,Harborside,JC104,Union St,JC051,40.719252,-74.034234,40.718211,-74.083639


In [14]:
# cutting to csv for future work
final_cut.to_csv('merged_csv_file/new_format_merge.csv')

In [15]:
# using glob to pull the other files -- this will miss one file, which is 202101 that i will bring in and merge after this round
path = "raw_csv_files/"
og_style_df = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "JC-20?[!1]*.csv"))))

In [16]:
len(og_style_df)

1095641

In [17]:
# using glob to pull the last file
path = "raw_csv_files/"
last_import_df = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "JC-202101-citibike-tripdata.csv"))))

In [18]:
len(last_import_df)

11624

In [19]:
#concatenate the last two imports
frames = [og_style_df, last_import_df]

result = pd.concat(frames)

In [20]:
print(len(result))

1107265


In [21]:
# grabbing the same columns as the other file
og_df_columns = result[['tripduration',
'starttime',
'stoptime',
'start station name',
'start station id',
'end station name',
'end station id',
'start station latitude',
'start station longitude',
'end station latitude',
'end station longitude'
]]

In [22]:
og_df_columns

,tripduration,starttime,stoptime,start station name,start station id,end station name,end station id,start station latitude,start station longitude,end station latitude,end station longitude
0,932,2018-01-01 02:06:17.5410,2018-01-01 02:21:50.0270,Exchange Place,3183,Newport Pkwy,3199,40.716247,-74.033459,40.728745,-74.032108
1,550,2018-01-01 12:06:18.0390,2018-01-01 12:15:28.4430,Exchange Place,3183,Newport Pkwy,3199,40.716247,-74.033459,40.728745,-74.032108
2,510,2018-01-01 12:06:56.9780,2018-01-01 12:15:27.8100,Exchange Place,3183,Newport Pkwy,3199,40.716247,-74.033459,40.728745,-74.032108
3,354,2018-01-01 14:53:10.1860,2018-01-01 14:59:05.0960,Exchange Place,3183,Morris Canal,3267,40.716247,-74.033459,40.712419,-74.038526
4,250,2018-01-01 17:34:30.1920,2018-01-01 17:38:40.9840,Exchange Place,3183,Harborside,3639,40.716247,-74.033459,40.719252,-74.034234
...,...,...,...,...,...,...,...,...,...,...,...
11619,2476,2021-01-31 20:16:05.4700,2021-01-31 20:57:22.0110,Warren St,3187,Montgomery St,3483,40.721124,-74.038051,40.719420,-74.050990
11620,147,2021-01-31 21:05:05.2090,2021-01-31 21:07:32.8660,Grove St PATH,3186,Warren St,3187,40.719586,-74.043117,40.721124,-74.038051
11621,660,2021-01-31 21:06:23.5880,2021-01-31 21:17:23.8420,Newark Ave,3211,Marin Light Rail,3276,40.721525,-74.046305,40.714584,-74.042817
11622,384,2021-01-31 21:16:37.8220,2021-01-31 21:23:02.1240,JC Medical Center,3205,Lafayette Park,3268,40.716540,-74.049638,40.713464,-74.062859


In [23]:
#concatenate the whole shebang
final_frames = [final_cut, og_df_columns]

total_results = pd.concat(final_frames)

In [24]:
len(total_results)

1543153

In [25]:
total_results.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station name',
       'start station id', 'end station name', 'end station id',
       'start station latitude', 'start station longitude',
       'end station latitude', 'end station longitude'],
      dtype='object')

In [26]:
# cutting to csv for future work
total_results.to_csv('merged_csv_file/total_results.csv')

In [27]:
start_lat_long = total_results[['start station latitude','start station longitude']]

In [28]:
start_lat_long_upd = start_lat_long.rename(columns = {"start station latitude":"station_lat",'start station longitude':"station_long"})

In [29]:
start_lat_long_uni = start_lat_long_upd.drop_duplicates()

In [30]:
start_lat_long_uni

,station_lat,station_long
0,40.735208,-74.046964
1,40.728744,-74.032108
2,40.733670,-74.062500
3,40.735208,-74.046963
6,40.721525,-74.046305
...,...,...
5324,40.711131,-74.078885
8712,40.697030,-74.096937
8715,40.698651,-74.082080
10107,40.692640,-74.088012


In [31]:
end_lat_long = total_results[['end station latitude','end station longitude']]

In [32]:
end_lat_long_upd = end_lat_long.rename(columns = {"end station latitude":"station_lat",'end station longitude':"station_long"})

In [33]:
end_lat_long_uni = end_lat_long_upd.drop_duplicates()

In [34]:
end_lat_long_uni

,station_lat,station_long
0,40.734786,-74.050444
1,40.714584,-74.042817
2,40.723659,-74.064194
3,40.735208,-74.046964
5,40.714145,-74.033552
...,...,...
51802,40.718170,-73.955201
6054,40.677343,-74.012751
16258,40.717730,-73.940510
9933,40.752694,-74.002353


In [35]:
merge_lat_long = pd.merge(start_lat_long_uni,end_lat_long_uni)

In [36]:
merge_lat_long

,station_lat,station_long
0,40.735208,-74.046964
1,40.733670,-74.062500
2,40.721525,-74.046305
3,40.734961,-74.059503
4,40.724294,-74.035483
...,...,...
152,40.711131,-74.078885
153,40.697030,-74.096937
154,40.698651,-74.082080
155,40.692640,-74.088012


In [37]:
merge_lat_long_test = merge_lat_long.head(5)

In [38]:
merge_lat_long['postal_code'] = ""
merge_lat_long

,station_lat,station_long,postal_code
0,40.735208,-74.046964,
1,40.733670,-74.062500,
2,40.721525,-74.046305,
3,40.734961,-74.059503,
4,40.724294,-74.035483,
...,...,...,...
152,40.711131,-74.078885,
153,40.697030,-74.096937,
154,40.698651,-74.082080,
155,40.692640,-74.088012,


In [39]:
import requests
import json
from config import key

for index,row in merge_lat_long.iterrows():
    
    station_lat = row["station_lat"]
    station_long = row["station_long"]


# Build the endpoint URL
    target_url = f"http://api.geonames.org/findNearbyPostalCodesJSON?lat={station_lat}&lng={station_long}&username={key}"
    geo_data = requests.get(target_url).json()
    
# Extract latitude and longitude
    try:
        print(geo_data['postalCodes'][0]['postalCode'])
        merge_lat_long.loc[index,"postal_code"] = geo_data['postalCodes'][0]['postalCode']
    except (KeyError, IndexError):
        print(f"Record could not be found")

07310
07306
07302
07306
07310
07302
07302
07302
07302
07304
07304
07305
07306
07304
07304
07302
07302
07310
07302
07306
07306
07306
07307
07302
07310
07304
07302
07302
07306
07302
07306
07306
07307
07302
07302
07302
07302
07302
07302
07302
07307
07304
07302
07306
07306
07302
07302
07302
07302
07302
07307
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07030
07302
07030
07306
07306
07310
07302
07310
07302
07302
07304
07307
07302
07302
07304
07306
07030
07302
07030
07030
07302
07306
07302
07030
07302
07302
07302
07030
07030
07030
07302
07310
07306
07306
07307
07030
07302
07310
07030
07305
07302
07304
07306
07304
07304
07030
07302
07302
07030
07307
07030
07030
07302
07030
07302
07302
07302
07304
07307
07306
07302
07306
07030
07030
07030
07030
07305
07030
07030
07305
07030
07030
07304
07302
07305
07305
07304
07306
07305
07305
07305
07305
07305


In [40]:
merge_lat_long

,station_lat,station_long,postal_code
0,40.735208,-74.046964,07310
1,40.733670,-74.062500,07306
2,40.721525,-74.046305,07302
3,40.734961,-74.059503,07306
4,40.724294,-74.035483,07310
...,...,...,...
152,40.711131,-74.078885,07305
153,40.697030,-74.096937,07305
154,40.698651,-74.082080,07305
155,40.692640,-74.088012,07305


In [41]:
# cutting to csv for future work
merge_lat_long.to_csv('merged_csv_file/lat_long_zip.csv')

In [42]:
# attempting to get ZIP Code by trip
total_results_zip = total_results[['tripduration','starttime','stoptime','start station latitude','start station longitude']]

In [43]:
xxx = total_results_zip.rename(columns = {'start station latitude':'station_lat','start station longitude':'station_long'})

In [44]:
xxx

,tripduration,starttime,stoptime,station_lat,station_long
0,420.0,2021-02-03 23:11:28,2021-02-03 23:18:28,40.735208,-74.046964
1,1355.0,2021-02-27 16:34:05,2021-02-27 16:56:40,40.728744,-74.032108
2,381.0,2021-02-26 23:16:04,2021-02-26 23:22:25,40.733670,-74.062500
3,1459.0,2021-02-24 16:51:50,2021-02-24 17:16:09,40.735208,-74.046963
4,30.0,2021-02-24 20:44:16,2021-02-24 20:44:46,40.735208,-74.046963
...,...,...,...,...,...
11619,2476.0,2021-01-31 20:16:05.4700,2021-01-31 20:57:22.0110,40.721124,-74.038051
11620,147.0,2021-01-31 21:05:05.2090,2021-01-31 21:07:32.8660,40.719586,-74.043117
11621,660.0,2021-01-31 21:06:23.5880,2021-01-31 21:17:23.8420,40.721525,-74.046305
11622,384.0,2021-01-31 21:16:37.8220,2021-01-31 21:23:02.1240,40.716540,-74.049638


In [45]:
new_df = pd.merge(xxx, merge_lat_long, how='left', on=['station_lat','station_long'])

In [46]:
new_df

,tripduration,starttime,stoptime,station_lat,station_long,postal_code
0,420.0,2021-02-03 23:11:28,2021-02-03 23:18:28,40.735208,-74.046964,07310
1,1355.0,2021-02-27 16:34:05,2021-02-27 16:56:40,40.728744,-74.032108,NaN
2,381.0,2021-02-26 23:16:04,2021-02-26 23:22:25,40.733670,-74.062500,07306
3,1459.0,2021-02-24 16:51:50,2021-02-24 17:16:09,40.735208,-74.046963,NaN
4,30.0,2021-02-24 20:44:16,2021-02-24 20:44:46,40.735208,-74.046963,NaN
...,...,...,...,...,...,...
1543148,2476.0,2021-01-31 20:16:05.4700,2021-01-31 20:57:22.0110,40.721124,-74.038051,07302
1543149,147.0,2021-01-31 21:05:05.2090,2021-01-31 21:07:32.8660,40.719586,-74.043117,07302
1543150,660.0,2021-01-31 21:06:23.5880,2021-01-31 21:17:23.8420,40.721525,-74.046305,07302
1543151,384.0,2021-01-31 21:16:37.8220,2021-01-31 21:23:02.1240,40.716540,-74.049638,07302


In [47]:
rslt_df = new_df[new_df['postal_code'].isna()]
rslt_df_2 = rslt_df[['station_lat','station_long']]
rslt_df_3 = rslt_df_2.drop_duplicates()
zzz = rslt_df_3.reset_index(drop=True)
zzz['postal_code'] = ""
zzz

,station_lat,station_long,postal_code
0,40.728744,-74.032108,
1,40.735208,-74.046963,
2,40.721525,-74.046304,
3,40.712418,-74.038525,
4,40.714358,-74.066610,
5,40.723658,-74.064194,
6,40.742677,-74.051788,
7,40.748715,-74.040443,
8,40.724294,-74.035482,
9,40.730897,-74.063912,


In [48]:
for index,row in zzz.iterrows():
    
    station_lat = row["station_lat"]
    station_long = row["station_long"]


# Build the endpoint URL
    target_url = f"http://api.geonames.org/findNearbyPostalCodesJSON?lat={station_lat}&lng={station_long}&username={key}"
    geo_data = requests.get(target_url).json()
    
# Extract latitude and longitude
    try:
        print(geo_data['postalCodes'][0]['postalCode'])
        zzz.loc[index,"postal_code"] = geo_data['postalCodes'][0]['postalCode']
        print(f"Record found at {station_lat},{station_long}")
    except (KeyError, IndexError):
        print(f"Record could not be found")

07310
Record found at 40.728744,-74.032108
07310
Record found at 40.735208,-74.046963
07302
Record found at 40.721525,-74.046304
07302
Record found at 40.712418,-74.038525
07304
Record found at 40.714358,-74.06661
07302
Record found at 40.723658,-74.064194
07307
Record found at 40.742677,-74.051788
07307
Record found at 40.748715,-74.040443
07310
Record found at 40.724294,-74.035482
07306
Record found at 40.730897,-74.063912
07306
Record found at 40.725339,-74.067622
07302
Record found at 40.719251,-74.034234
07302
Record found at 40.714145,-74.033551
07302
Record found at 40.721123,-74.03805
07306
Record found at 40.727551,-74.07106
07306
Record found at 40.737603,-74.052478
07306
Record found at 40.731168,-74.057573
07302
Record found at 40.723331,-74.045952
07304
Record found at 40.722103,-74.071455
07304
Record found at 40.719282,-74.071261
07310
Record found at 40.727223,-74.033758
07302
Record found at 40.724176,-74.050656
07304
Record found at 40.713463,-74.062858
07302
Record f

In [49]:
zzz

,station_lat,station_long,postal_code
0,40.728744,-74.032108,07310
1,40.735208,-74.046963,07310
2,40.721525,-74.046304,07302
3,40.712418,-74.038525,07302
4,40.714358,-74.066610,07304
5,40.723658,-74.064194,07302
6,40.742677,-74.051788,07307
7,40.748715,-74.040443,07307
8,40.724294,-74.035482,07310
9,40.730897,-74.063912,07306


In [50]:
new_new_zip = pd.concat([zzz, merge_lat_long])

In [51]:
new_new_zip

,station_lat,station_long,postal_code
0,40.728744,-74.032108,07310
1,40.735208,-74.046963,07310
2,40.721525,-74.046304,07302
3,40.712418,-74.038525,07302
4,40.714358,-74.066610,07304
...,...,...,...
152,40.711131,-74.078885,07305
153,40.697030,-74.096937,07305
154,40.698651,-74.082080,07305
155,40.692640,-74.088012,07305


In [52]:
new_new_df = pd.merge(xxx, new_new_zip, how='left', on=['station_lat','station_long'])

In [53]:
new_new_df

,tripduration,starttime,stoptime,station_lat,station_long,postal_code
0,420.0,2021-02-03 23:11:28,2021-02-03 23:18:28,40.735208,-74.046964,07310
1,1355.0,2021-02-27 16:34:05,2021-02-27 16:56:40,40.728744,-74.032108,07310
2,381.0,2021-02-26 23:16:04,2021-02-26 23:22:25,40.733670,-74.062500,07306
3,1459.0,2021-02-24 16:51:50,2021-02-24 17:16:09,40.735208,-74.046963,07310
4,30.0,2021-02-24 20:44:16,2021-02-24 20:44:46,40.735208,-74.046963,07310
...,...,...,...,...,...,...
1543148,2476.0,2021-01-31 20:16:05.4700,2021-01-31 20:57:22.0110,40.721124,-74.038051,07302
1543149,147.0,2021-01-31 21:05:05.2090,2021-01-31 21:07:32.8660,40.719586,-74.043117,07302
1543150,660.0,2021-01-31 21:06:23.5880,2021-01-31 21:17:23.8420,40.721525,-74.046305,07302
1543151,384.0,2021-01-31 21:16:37.8220,2021-01-31 21:23:02.1240,40.716540,-74.049638,07302


In [54]:
# cutting to csv for future work
new_new_df.to_csv('merged_csv_file/postal_code_results.csv')